In [1]:
# import libraries
import os
import pandas as pd
import json
import gzip
import fasttext 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd
import re
import numpy as np
import sklearn

In [4]:
test = pd.read_csv('../../src/data/schemafiltereddata/TrainTestTables/archiv/test_tables_seq_9000.csv')

In [5]:
test

,Unnamed: 0,text,name,class
0,0,Comedy Improv at Arts Garage 2020-12-10 Shows ...,name,Event
1,1,2020-12-10T21:30:00-05:00 2020-12-10T21:30:0...,enddate,Event
2,2,"{'name': 'Arts Garage'},{'name': 'Bob Carter&#...",location,Event
3,3,"{'name': 'Mod 27 Improv', 'telephone': '561-22...",organizer,Event
4,4,My Front Door Lord Have Mercy Sold Out Wedding...,name,MusicRecording
...,...,...,...,...
49599,49599,{'url': 'https://eutrade-shop.com/epages/f7e4b...,offers,Product
49600,49600,"4,5,5,3,5,5,5,4,3,5,3,5,5,5,3,5,1,5,5,4,3,3,5,...",category,Product
49601,49601,http://schema.org/NewCondition http://schema.o...,itemcondition,Product
49602,49602,"4313042590504.0,4313042595028.0,4313042595066....",gtin13,Product


In [8]:
test.drop(test[(test['name']=='inalbum') &(test['class']=='MusicRecording')].index, inplace=True)
test.drop(test[(test['name']=='performer') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='eventattendancemode') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='eventstatus') &(test['class']=='Event')].index, inplace=True)
test["label_complete"] = test["name"] + test["class"]
num_columns = len(set.union(set(test.label_complete.unique())))
test

,Unnamed: 0,text,name,class,label_complete
0,0,Comedy Improv at Arts Garage 2020-12-10 Shows ...,name,Event,nameEvent
1,1,2020-12-10T21:30:00-05:00 2020-12-10T21:30:0...,enddate,Event,enddateEvent
2,2,"{'name': 'Arts Garage'},{'name': 'Bob Carter&#...",location,Event,locationEvent
3,3,"{'name': 'Mod 27 Improv', 'telephone': '561-22...",organizer,Event,organizerEvent
4,4,My Front Door Lord Have Mercy Sold Out Wedding...,name,MusicRecording,nameMusicRecording
...,...,...,...,...,...
49599,49599,{'url': 'https://eutrade-shop.com/epages/f7e4b...,offers,Product,offersProduct
49600,49600,"4,5,5,3,5,5,5,4,3,5,3,5,5,5,3,5,1,5,5,4,3,3,5,...",category,Product,categoryProduct
49601,49601,http://schema.org/NewCondition http://schema.o...,itemcondition,Product,itemconditionProduct
49602,49602,"4313042590504.0,4313042595028.0,4313042595066....",gtin13,Product,gtin13Product


In [9]:
test.drop(['Unnamed: 0','class'], axis=1, inplace=True)
test['text'] = test.text.apply(lambda x: re.sub("[^0-9a-zA-Z-@]+", " ", x))

In [10]:
labels_new = {'worstratingProduct': 0,
 'acceptsreservationsRestaurant': 1,
 'additionalnamePerson': 2,
 'additionalpropertyProduct': 3,
 'addressHotel': 4,
 'addressLocalBusiness': 5,
 'addressPerson': 6,
 'addressProduct': 7,
 'addressRestaurant': 8,
 'addresslocalityLocalBusiness': 9,
 'addressregionLocalBusiness': 10,
 'affiliationPerson': 11,
 'aggregateratingBook': 12,
 'aggregateratingCreativeWork': 13,
 'aggregateratingHotel': 14,
 'aggregateratingLocalBusiness': 15,
 'aggregateratingProduct': 16,
 'aggregateratingRecipe': 17,
 'aggregateratingRestaurant': 18,
 'alternatenameProduct': 19,
 'alternativeheadlineCreativeWork': 20,
 'areaservedLocalBusiness': 21,
 'articlebodyCreativeWork': 22,
 'audienceProduct': 23,
 'authorBook': 24,
 'authorCreativeWork': 25,
 'authorProduct': 26,
 'authorRecipe': 27,
 'availabilityProduct': 28,
 'bestratingProduct': 29,
 'birthdatePerson': 30,
 'birthplacePerson': 31,
 'bookeditionBook': 32,
 'bookformatBook': 33,
 'brandProduct': 34,
 'breadcrumbProduct': 35,
 'byartistMusicAlbum': 36,
 'byartistMusicRecording': 37,
 'categoryProduct': 38,
 'citystatezipLocalBusiness': 39,
 'colorProduct': 40,
 'commentcountCreativeWork': 41,
 'conditionProduct': 42,
 'contactpointLocalBusiness': 43,
 'contactpointPerson': 44,
 'cookingmethodRecipe': 45,
 'cooktimeRecipe': 46,
 'copyrightholderCreativeWork': 47,
 'copyrightyearCreativeWork': 48,
 'creatorCreativeWork': 49,
 'datecreatedCreativeWork': 50,
 'datecreatedMusicRecording': 51,
 'datemodifiedCreativeWork': 52,
 'datemodifiedRecipe': 53,
 'datepublishedBook': 54,
 'datepublishedCreativeWork': 55,
 'datepublishedMusicRecording': 56,
 'datepublishedProduct': 57,
 'datepublishedRecipe': 58,
 'deathdatePerson': 59,
 'depthProduct': 60,
 'disambiguatingdescriptionProduct': 61,
 'doortimeEvent': 62,
 'durationEvent': 63,
 'durationMusicRecording': 64,
 'emailHotel': 65,
 'emailLocalBusiness': 66,
 'emailPerson': 67,
 'enddateEvent': 68,
 'episodenumberTVEpisode': 69,
 'worksforPerson': 70,
 'familynamePerson': 71,
 'faxnumberLocalBusiness': 72,
 'faxnumberPerson': 73,
 'genderPerson': 74,
 'genreBook': 75,
 'genreCreativeWork': 76,
 'genreMusicRecording': 77,
 'geoHotel': 78,
 'geoLocalBusiness': 79,
 'geoPlace': 80,
 'geoProduct': 81,
 'geoRestaurant': 82,
 'givennamePerson': 83,
 'gtin12Product': 84,
 'gtin13Product': 85,
 'gtin14Product': 86,
 'gtin8Product': 87,
 'gtinProduct': 88,
 'hasmapLocalBusiness': 89,
 'hasmenuRestaurant': 90,
 'headlineCreativeWork': 91,
 'headlineRecipe': 92,
 'heightPerson': 93,
 'heightProduct': 94,
 'homelocationPerson': 95,
 'identifierProduct': 96,
 'worklocationPerson': 97,
 'ingredientsRecipe': 98,
 'inlanguageBook': 99,
 'inlanguageCreativeWork': 100,
 'interactioncountCreativeWork': 101,
 'interactionstatisticCreativeWork': 102,
 'interactiontypeCreativeWork': 103,
 'isbnBook': 104,
 'ispartofRecipe': 105,
 'isrelatedtoProduct': 106,
 'issimilartoProduct': 107,
 'itemconditionProduct': 108,
 'itemlistelementProduct': 109,
 'jobtitlePerson': 110,
 'keywordsCreativeWork': 111,
 'keywordsRecipe': 112,
 'knowslanguagePerson': 113,
 'legalnameLocalBusiness': 114,
 'locationEvent': 115,
 'locationLocalBusiness': 116,
 'mainentityofpageCreativeWork': 117,
 'mainentityofpagePerson': 118,
 'mainentityofpageProduct': 119,
 'mainentityofpageRecipe': 120,
 'makesofferPerson': 121,
 'manufacturerProduct': 122,
 'materialProduct': 123,
 'memberofPerson': 124,
 'menuRestaurant': 125,
 'modelProduct': 126,
 'mpnProduct': 127,
 'nameBook': 128,
 'nameCreativeWork': 129,
 'nameEvent': 130,
 'nameHotel': 131,
 'nameLocalBusiness': 132,
 'nameMusicAlbum': 133,
 'nameMusicRecording': 134,
 'namePlace': 135,
 'nameProduct': 136,
 'nameRecipe': 137,
 'nameRestaurant': 138,
 'nameTVEpisode': 139,
 'nationalityPerson': 140,
 'numberofpagesBook': 141,
 'numtracksMusicAlbum': 142,
 'nutritionRecipe': 143,
 'offerdetailsProduct': 144,
 'offersBook': 145,
 'offersCreativeWork': 146,
 'offersProduct': 147,
 'openinghoursLocalBusiness': 148,
 'openinghoursRestaurant': 149,
 'openinghoursspecificationLocalBusiness': 150,
 'openinghoursspecificationPlace': 151,
 'openinghoursspecificationRestaurant': 152,
 'organizerEvent': 153,
 'partofseriesTVEpisode': 154,
 'paymentacceptedLocalBusiness': 155,
 'widthProduct': 156,
 'performersEvent': 157,
 'performtimeRecipe': 158,
 'postalcodeLocalBusiness': 159,
 'preptimeRecipe': 160,
 'priceProduct': 161,
 'pricecurrencyProduct': 162,
 'pricerangeHotel': 163,
 'pricerangeLocalBusiness': 164,
 'pricerangeRestaurant': 165,
 'publisherBook': 166,
 'publisherCreativeWork': 167,
 'publisherRecipe': 168,
 'ratingvalueProduct': 169,
 'recipecategoryRecipe': 170,
 'recipecuisineRecipe': 171,
 'recipeingredientRecipe': 172,
 'recipeinstructionsRecipe': 173,
 'recipeyieldRecipe': 174,
 'releasedateProduct': 175,
 'reviewLocalBusiness': 176,
 'reviewProduct': 177,
 'reviewRecipe': 178,
 'reviewcountProduct': 179,
 'reviewsProduct': 180,
 'sameasLocalBusiness': 181,
 'sameasPerson': 182,
 'sameasPlace': 183,
 'servescuisineRestaurant': 184,
 'shop-currencyProduct': 185,
 'starratingHotel': 186,
 'streetaddressLocalBusiness': 187,
 'suitablefordietRecipe': 188,
 'telephoneHotel': 189,
 'telephoneLocalBusiness': 190,
 'telephonePerson': 191,
 'telephonePlace': 192,
 'telephoneRestaurant': 193,
 'titleProduct': 194,
 'totaltimeRecipe': 195,
 'trackMusicAlbum': 196,
 'typicalagerangeEvent': 197,
 'versionCreativeWork': 198,
 'weightPerson': 199,
 'weightProduct': 200,
 'founderLocalBusiness': 201
 #'eventattendancemodeEvent': 201,
 #'performerEvent': 202,
 #'inalbumMusicRecording': 203,
 #'eventstatusEvent': 204
 }

In [11]:
test

,text,name,label_complete
0,Comedy Improv at Arts Garage 2020-12-10 Shows ...,name,nameEvent
1,2020-12-10T21 30 00-05 00 2020-12-10T21 30 00-...,enddate,enddateEvent
2,name Arts Garage name Bob Carter 8217 s Actor...,location,locationEvent
3,name Mod 27 Improv telephone 561-223-1651 ema...,organizer,organizerEvent
4,My Front Door Lord Have Mercy Sold Out Wedding...,name,nameMusicRecording
...,...,...,...
49599,url https eutrade-shop com epages f7e4b224-a2...,offers,offersProduct
49600,4 5 5 3 5 5 5 4 3 5 3 5 5 5 3 5 1 5 5 4 3 3 5 ...,category,categoryProduct
49601,http schema org NewCondition http schema org N...,itemcondition,itemconditionProduct
49602,4313042590504 0 4313042595028 0 4313042595066 ...,gtin13,gtin13Product


In [12]:
test['label'] = test.label_complete.map(labels_new)
test.drop(['label_complete', 'name'], axis=1, inplace=True)
test

,text,label
0,Comedy Improv at Arts Garage 2020-12-10 Shows ...,130
1,2020-12-10T21 30 00-05 00 2020-12-10T21 30 00-...,68
2,name Arts Garage name Bob Carter 8217 s Actor...,115
3,name Mod 27 Improv telephone 561-223-1651 ema...,153
4,My Front Door Lord Have Mercy Sold Out Wedding...,134
...,...,...
49599,url https eutrade-shop com epages f7e4b224-a2...,147
49600,4 5 5 3 5 5 5 4 3 5 3 5 5 5 3 5 1 5 5 4 3 3 5 ...,38
49601,http schema org NewCondition http schema org N...,108
49602,4313042590504 0 4313042595028 0 4313042595066 ...,85


In [14]:
test.to_csv('../../src/data/schemafiltereddata/TrainTestTables/archiv/test_tables_9000_prepped.csv')